In [1]:
import timm
from PIL import Image
import torch
from torchvision import transforms
from torch.nn import functional as F
import timm.models.mlp_mixer
import numpy as np
import exchange_tensor_array as exchange
import matplotlib.pyplot as plt

from pathlib import Path
from torchvision.datasets.utils import download_url
import json

import copy
#モデル作成
model = timm.create_model("gmlp_s16_224", pretrained=True)
model.eval()
print("model")

transform = transforms.Compose(
    [
        transforms.Resize(256),  # (256, 256) で切り抜く。
        transforms.CenterCrop(224),  # 画像の中心に合わせて、(224, 224) で切り抜く
        transforms.ToTensor(),  # テンソルにする。
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
        ),  # 標準化する。
    ]
)

transform_beta = transforms.Compose(
    [
        transforms.Resize(256),  # (256, 256) で切り抜く。
        transforms.CenterCrop(224),  # 画像の中心に合わせて、(224, 224) で切り抜く
    ]
)

img = Image.open("cat.jpg")
inputs = transform(img)
inputs = inputs.unsqueeze(0)
#plt.imshow(inputs[0].permute(1,2,0))
copy_inputs = copy.deepcopy(inputs)

/usr/local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


model


In [2]:
#クラス版テスト
import cluster_ablation as ca
exp = ca.cluster_ablation(model)
shap_c = exp.calc_shapley_save_img("cat.jpg", "cat_reslut.jpg")

In [ ]:
shap_c

In [ ]:
y = [i[0] for i in p]
x = [i[1] for i in p]
cover_img = np.zeros((224, 224, 4), dtype=np.uint8)
for i in range(224):
    for j in range(224):
        cover_img[i, j, 3] = 0
shap_max = max(shap_c)
for i, j, c in zip(y, x, pred):
    if shap[pred]
    cover_img[i, j, 0] = 255
    cover_img[i, j, 3] = 255
cover_img = Image.fromarray(cover_img)
base_img = transform_beta(img)
base_img.paste(cover_img, (0, 0), cover_img)
plt.imshow(base_img)

In [ ]:
plt.imshow(inputs_c[39][0].permute(1,2,0))

In [ ]:
shap_c

In [ ]:
output0 = model(inputs)
mid0 = model.blocks[0].block_output
mid0 = exchange.exchange_tensor_to_array(mid0)
plt.imshow(mid0)

In [ ]:
from sklearn.cluster import DBSCAN
y, x = np.where((abs(mid0) > 2.7235))
l = len(y)
p = [ [y[i], x[i]] for i in range(l)]
print(l)
tmp = np.zeros((224, 224))
for u, v in p:
    tmp[u, v] = 10
exchange.show_heatmap_with_colorbar(tmp)


In [ ]:
for i in range(0, 50, 2):
    y, x = np.where((abs(mid0) > i * 0.1))
    print(i * 0.1, len(y))
    tmp = np.zeros((224, 224))
    for u, v in zip(y, x):
        tmp[u, v] = 10
    exchange.show_heatmap_with_colorbar(tmp)

In [ ]:
y, x = np.where((abs(mid0) > 2.7235))
l = len(y)
p = [ [y[i], x[i]] for i in range(l)]
print(l)

In [ ]:
type(y)

In [ ]:
res = []
for i in range(1, 21):
    db = DBSCAN(eps=9, min_samples = i)
    pred = db.fit_predict(p)
    n_c = set(pred)
    fig, ax = plt.subplots()
    plt.scatter(x, y, s=100, c=pred, cmap="Blues")
    plt.colorbar()
    plt.xlim(0, 223)
    plt.ylim(223, 0)
    res.append((len(n_c), list(pred).count(-1)))
print(res)

In [ ]:
from sklearn.cluster import DBSCAN
db = DBSCAN(eps=9, min_samples=5)
pred = db.fit_predict(p)
n_c = set(pred)
fig, ax = plt.subplots()
plt.scatter(x, y, s=100, c=pred, cmap="Blues")
plt.colorbar()
plt.xlim(0, 223)
plt.ylim(223, 0)
len(set(pred))

In [ ]:
type(pred)

In [ ]:
for i in range(1, 21):
    db = DBSCAN(eps=8, min_samples=i)
    pred = db.fit_predict(p)
    n_c = set(pred)
    fig, ax = plt.subplots()
    plt.scatter(x, y, s = 100, c = pred, cmap="Blues")
    plt.colorbar()
    plt.xlim(0, 223)
    plt.ylim(223, 0)
    print(len(set(pred)))

In [ ]:
#Shaply値を愚直に
from scipy.spatial import ConvexHull, Delaunay
from collections import defaultdict
class shaply_from_first_mid_output():
    
    def __init__(self, model : timm.models.mlp_mixer.MlpMixer) -> None:
        self.model = model
        self.class_names = None
        self.get_classes()
        self.inputs = []
        self.values = []
        self.block_output = None
        self.block_output_exchanged = None
        
        
    def get_classes(self):
        if not Path("data/imagenet_class_index.json").exists():
            # ファイルが存在しない場合はダウンロードする。
            download_url("https://git.io/JebAs", "data", "imagenet_class_index.json")

        # クラス一覧を読み込む。
        with open("data/imagenet_class_index.json") as f:
            data = json.load(f)
            self.class_names = [x["ja"] for x in data]

    def base_model_output(self, _input : torch.Tensor) -> tuple:
        """画像をモデルに入力した際の一番スコアが高いラベルとそのスコアを返す"""
        base_output = self.model(_input)
        batch_probs = F.softmax(base_output, dim=1)
        batch_probs, batch_indices = batch_probs.sort(dim=1, descending=True)

        return (self.class_names[batch_indices[0][0]], batch_probs[0][0].item())

    def in_hull_(self, point : list, hull : list):
        """点群hullからなる凸包内に点pointが入っているかを判定"""
        if not isinstance(hull, Delaunay):
            hull = Delaunay(hull)
        return hull.find_simplex(point) >= 0

"""
    def cal_paras_filitering(self):
        #にぶたんとかでフィルタリングのパラメータを求める
        return フィルタリングのパラメータ
    
    def cal_paras_DESCAN(self):
        #にぶたんとかでDBSCANのパラメータを求める
        return DBSCANのパラメータ
    
    def clustering(self):
        #DBSCAMを行って、クラスタのラベルを返す
        return pred
"""

    def calc_shaply(img : torch.Tensor):
        
        
        
        
        
        #クラスタごとに点群を求める。各点群の座標の最大値最小値も記録しておく
        clusters = defaultdict(lambda : [])
    
    

In [ ]:
#Shaply値を愚直に
from scipy.spatial import ConvexHull, Delaunay
from collections import defaultdict
from math import factorial
def in_hull_(point : list, hull : list):
        """点群hullからなる凸包内に点pointが入っているかを判定"""
        if not isinstance(hull, Delaunay):
            hull = Delaunay(hull)
        return hull.find_simplex(point) >= 0
    
    
def get_classes():
    if not Path("data/imagenet_class_index.json").exists():
        # ファイルが存在しない場合はダウンロードする。
        download_url("https://git.io/JebAs", "data", "imagenet_class_index.json")

    # クラス一覧を読み込む。
    with open("data/imagenet_class_index.json") as f:
        data = json.load(f)
        class_names = [x["ja"] for x in data]
    return class_names

class_names = get_classes()

def base_model_output(_input : torch.Tensor) -> tuple:
    """画像をモデルに入力した際の一番スコアが高いラベルとそのスコアを返す"""
    base_output = model(_input)
    batch_probs = F.softmax(base_output, dim=1)
    batch_probs, batch_indices = batch_probs.sort(dim=1, descending=True)
    
    return (class_names[batch_indices[0][0]], batch_probs[0][0].item())

def calc_shaply(img : torch.Tensor, y : np.ndarray, x : np.ndarray, pred : np.ndarray):
    
    inputs = []
    values = []
    
    
    #クラスタ数
    n_c = len(set(pred)) - 1
    #shaply値格納
    shap = [0] * n_c
    #各マスクされた画像に対するスコア格納
    probs = [0] * pow(2, n_c)
    #マスクする際のバックアップ
    input_backup = defaultdict(lambda : [])
    
    
    #元々のスコアとクラス名とそのインデックスを取得。便宜上ここにおいておく
    base_class_name, base_prob = base_model_output(img)
    base_class_index = class_names.index(base_class_name)
    
    num = []
    
    
    #クラスタごとに点群を求める。各点群の座標の最大値最小値も記録しておく
    clusters = defaultdict(lambda : [])
    left = defaultdict(lambda : 1000)
    right = defaultdict(lambda : -1000)
    up = defaultdict(lambda : 1000)
    down = defaultdict(lambda : -1000)
    
    for v, u, c in zip(y, x, pred):
        if c == -1:
            continue
        clusters[c].append((v, u))
        left[c] = min(left[c], u)
        right[c] = max(right[c], u)
        up[c] = min(up[c], v)
        down[c] = max(down[c], v)
        

    #全組み合わせを総当たり
    for i in range(2 ** n_c):
        flag = [True] * n_c #マスクされていたらFalse
        input_backup = defaultdict(lambda : [])
        for c in range(n_c):
            if i >> c & 1:
                flag[c] = False
                #マスクする
                for v in range(up[c], down[c] + 1):
                    for u in range(left[c], right[c] + 1):
                        if in_hull_([v, u], clusters[c]):
                            r, b, g = img[0, 0, v, u].item(), img[0, 1, v, u].item(), img[0, 2, v, u].item()
                            input_backup[(v, u)] = [r, b, g]
                            img[0, 0, v, u] = 0
                            img[0, 1, v, u] = 0
                            img[0, 2, v, u] = 0

        
        #モデルに流してスコアを得る
        tmp_output = model(img)
        tmp_batch_probs = F.softmax(tmp_output, dim=1)
        tmp_prob = tmp_batch_probs[0][base_class_index].item()
        
        probs[i] = tmp_prob
        
        """
        for c in range(n_c):
            if flag[c]:
                shap[c] += tmp_prob
        """
        
        inputs.append(copy.deepcopy(img))
        values.append(tmp_prob)
        
        #入力画像を元に戻す
        for key, val in input_backup.items():
            v, u = key
            r, b, g = val
            img[0, 0, v, u] = r
            img[0, 1, v, u] = b
            img[0, 2, v, u] = g
    
    for i in range(2 ** n_c):
        count = 0
        p = []
        for j in range(n_c):
            if i >> j & 1:
                count += 1
            else:
                p.append(j)
        for j in p:
            tmp = factorial(count) * factorial(n_c - count - 1) * (-probs[i + pow(2, j)] + probs[i]) / factorial(n_c)
            shap[j] += tmp
            print(tmp,probs[i + pow(2, j)],probs[i])
            num.append(i + pow(2, j))
            num.append(i)
    
    
    return shap, inputs, values, num
        
    
                            
def meguru_bisect_for_filitering(mid_output : np.ndarray, border = 300) -> float:
    """中間層の出力をフィルタリングする閾値をピックアップする特徴量の数を基に二分探索で求める"""
    ok = 0
    ng = max(abs(mid_output.max()), abs(mid_output.min())) + 1
    while abs(ok - ng) > 0.01:
        mid = (ok + ng) / 2
        print(mid)
        if len(np.where(abs(mid_output) > mid)[0]) >= border:
            ok = mid
        else:
            ng = mid
    return ok
                
                
                
def calc_shaply_beta(img : torch.Tensor, y : np.ndarray, x : np.ndarray, pred : np.ndarray):
    
    inputs = []
    values = []
    
    
    #クラスタ数
    n_c = len(set(pred))
    #shaply値格納
    shap = [0] * n_c
    #各マスクされた画像に対するスコア格納
    probs = [0] * pow(2, n_c)
    #マスクする際のバックアップ
    input_backup = defaultdict(lambda : [])
    
    
    #元々のスコアとクラス名とそのインデックスを取得。便宜上ここにおいておく
    base_class_name, base_prob = base_model_output(img)
    base_class_index = class_names.index(base_class_name)
    
    num = []
    
    
    #クラスタごとに点群を求める。各点群の座標の最大値最小値も記録しておく
    clusters = defaultdict(lambda : [])
    left = defaultdict(lambda : 1000)
    right = defaultdict(lambda : -1000)
    up = defaultdict(lambda : 1000)
    down = defaultdict(lambda : -1000)
    
    for v, u, c in zip(y, x, pred):
        if c == -1:
            continue
        clusters[c].append((v, u))
        left[c] = min(left[c], u)
        right[c] = max(right[c], u)
        up[c] = min(up[c], v)
        down[c] = max(down[c], v)
    
    
    
    #各画素ごとにどのクラスタの凸包に属するかを判定
    masks = defaultdict(lambda : [])
    mask_flag = [ [True] * 224 for _ in range(224)]
    
    for c in range(n_c - 1):
        for i in range(up[c], down[c] + 1):
            for j in range(left[c], right[c] + 1):
                if in_hull_([i, j], clusters[c]):
                    masks[c].append((i, j))
                    mask_flag[i][j] = False
    for i in range(224):
        for j in range(224):
            if mask_flag[i][j]:
                #img[0, 0, i, j] = 255
                #img[0, 1, i, j] = 255
                #img[0, 2, i, j] = 255
                masks[n_c - 1].append((i, j))
                
    
    """
    for i in range(224):
        for j in range(224):
            flag = True
            for c in range(n_c):
                if in_hull_((i, j), clusters[c]):
                    masks[c].append((i, j))
                    flag = False
                    break
            if flag:
                masks[n_c - 1].append((i, j))
    """
        

    #全組み合わせを総当たり
    for i in range(2 ** n_c):
        input_backup = defaultdict(lambda : [])
        for c in range(n_c):
            if i >> c & 1:
                #マスクする
                for y, x in masks[c]:
                    r, b, g = img[0, 0, y, x].item(), img[0, 1, y, x].item(), img[0, 2, y, x].item()
                    input_backup[(y, x)] = [r, b, g]
                    img[0, 0, y, x] = 0
                    img[0, 1, y, x] = 0
                    img[0, 2, y, x] = 0

        
        #モデルに流してスコアを得る
        tmp_output = model(img)
        tmp_batch_probs = F.softmax(tmp_output, dim=1)
        tmp_prob = tmp_batch_probs[0][base_class_index].item()
        
        probs[i] = tmp_prob
        
        inputs.append(copy.deepcopy(img))
        values.append(tmp_prob)
        
        #入力画像を元に戻す
        for key, val in input_backup.items():
            v, u = key
            r, b, g = val
            img[0, 0, v, u] = r
            img[0, 1, v, u] = b
            img[0, 2, v, u] = g
    
    for i in range(2 ** n_c):
        count = 0
        p = []
        for j in range(n_c):
            if i >> j & 1:
                count += 1
            else:
                p.append(j)
        for j in p:
            tmp = factorial(count) * factorial(n_c - count - 1) * (-probs[i + pow(2, j)] + probs[i]) / factorial(n_c)
            shap[j] += tmp
            #print(tmp,probs[i + pow(2, j)],probs[i])
            num.append(i + pow(2, j))
            num.append(i)
    
    
    return shap, inputs, values, num, masks, clusters, left, right, up, down

In [ ]:
#クラス版テスト
import cluster_ablation as ca
exp = ca.cluster_ablation(model)
shap_c, inputs_c, values_c, masks_c, p_c, pred_c, mask_flag_c = exp.calc_shapley(inputs)

In [ ]:
#クラス版テスト
import cluster_ablation as ca
exp = ca.cluster_ablation(model)
shap_c, inputs_c, values_c, masks_c, p_c, pred_c, mask_flag_c, clus_c, left_c, rihgt_c, up_c, down_c = exp.calc_shapley(inputs)
shap, _inputs, values, num, masks, clusters, left, right, up, down = calc_shaply_beta(inputs, y, x, pred)

In [ ]:
shap == shap_c

In [ ]:
shap, _inputs, values, num, masks, clusters, left, right, up, down = calc_shaply_beta(inputs, y, x, pred)

In [ ]:
a = [[y[i], x[i], pred[i]] for i in range(len(y))]
b = [[p_c[i][0], p_c[i][1], pred_c[i]] for i in range(len(p))]
a == b

In [ ]:
mask = defaultdict(lambda : [])
mask_f = [ [-1] * 224 for _ in range(224)]
clus = defaultdict(lambda : [])
for i, j, c in zip(y, x, pred):
    clus[c].append((i, j))
l = len(pred)
for c in range(l - 1):
    for i in range(224):
        for j in range(224):
            if in_hull_([i, j], clus[c]):
                mask[c].append((i, j))
                mask_f[i][j] = c
                break

In [ ]:
mask_flag_c

In [ ]:
masks_c

In [ ]:
a = [ len(i) for i in masks.values()]
b = [len(i) for i in masks_c.values()]
print(a,b)

In [ ]:
probs_c_sorted = sorted(probs_c)
values_sorted = sorted(values)
print(len(probs_c_sorted), len(values_sorted))

In [ ]:
count = 0
for i, j in zip(probs_c_sorted, values_sorted):
    if i != j:
        count += 1
print(count)

In [ ]:
shap

In [ ]:
for k in masks.keys():
    print("len mask[{}] : {}".format(k, len(masks[k])))

In [ ]:
sorted(values)

In [ ]:
masks.keys()

In [ ]:
shap

In [ ]:
base_class_name, base_prob = base_model_output(inputs)
base_class_index = class_names.index(base_class_name)
tmp_output = model(_inputs[2**5])
tmp_batch_probs = F.softmax(tmp_output, dim=1)
tmp_prob = tmp_batch_probs[0][base_class_index].item()

In [ ]:
tmp_prob

In [ ]:
plt.imshow(_inputs[0][0].permute(1,2,0))

In [ ]:
shap_c

In [ ]:
shap_max

In [ ]:
cover_img = np.zeros((224, 224, 4), dtype=np.uint8)
for i in range(224):
    for j in range(224):
        cover_img[i, j, 3] = 0
shap_max = max(shap_c[0][:-1])
for i, c in zip(shap_c[1], shap_c[2]):
    if c == -1 or shap_c[0][c] < shap_max / 2:
        continue
    y, x = i
    cover_img[y, x, 0] = 255 * shap_c[0][c] / shap_max
    cover_img[y, x, 3] = 255
cover_img = Image.fromarray(cover_img)
base_img = transform_beta(img)
base_img.paste(cover_img, (0, 0), cover_img)
plt.imshow(base_img)

In [ ]:
type(base_img)

In [ ]:
plt.imshow(img)

In [ ]:
plt.imshow(cover_img)

In [ ]:
base_img = transform_beta(img)

In [ ]:
plt.imshow(base_img)

In [ ]:
base_img.paste(cover_img, (0, 0), cover_img)

In [ ]:
plt.imshow(base_img)

In [ ]:
fig, ax = plt.subplots()
plt.scatter(x, y, s=100, c=pred, cmap="Blues")
plt.colorbar()
plt.xlim(0, 223)
plt.ylim(223, 0)

In [ ]:
#クラスタごとに点群を求める。各点群の座標の最大値最小値も記録しておく
clusters = defaultdict(lambda : [])
left = defaultdict(lambda : 1000)
right = defaultdict(lambda : -1000)
up = defaultdict(lambda : 1000)
down = defaultdict(lambda : -1000)

for v, u, c in zip(y, x, pred):
    if c == -1:
        continue
    clusters[c].append((v, u))
    left[c] = min(left[c], u)
    right[c] = max(right[c], u)
    up[c] = min(up[c], v)
    down[c] = max(down[c], v)

In [ ]:
for c in range(5):
    for i, j in clusters[c]:
        if not in_hull_([i, j], clusters[c]):
            print("not in ")

In [ ]:
for c in range(5):
    print(left[c], right[c], up[c],down[c])

In [ ]:
inputs.shape

In [ ]:
output = model(tmp)

In [ ]:
mid = model.blocks[0].block_output
mid = exchange.exchange_tensor_to_array(mid)

In [ ]:
exchange.show_heatmap_with_colorbar(mid)

In [ ]:
len(np.where(mid<-0.5)[0])

In [ ]:
tmp2 = torch.ones(1,3,224,224)
tmp2 *= 255
output2 = model(tmp2)
mid2 = model.blocks[0].block_output
mid2 = exchange.exchange_tensor_to_array(mid2)
exchange.show_heatmap_with_colorbar(mid2)

In [ ]:
len(np.where(abs(mid2) < 10)[0])

In [ ]:
mid2b = np.where(abs(mid2) < 25, 255, 0)
plt.imshow(mid2b)

In [ ]:
mid3 = mid + mid2 / 400

In [ ]:
exchange.show_heatmap_with_colorbar(mid3)